# **Project Description**

This notebook presents a sophisticated, two-stage Deep Learning (DL) framework for dynamic portfolio allocation, using real stock market data from a diverse universe of European companies.

**Project Objective**

The primary objective is to construct a long-only portfolio that maximizes the risk-adjusted return, measured by the Sharpe Ratio ($\text{SR}$), over a multi-period backtesting window.

I achieved this by moving beyond traditional Mean-Variance Optimization and leveraging the power of neural networks for end-to-end policy learning.

**Methodology**:

The Two-Stage Policy Network

The system is built on a two-stage sequential decision process:

**Stage 1: CNN-based Alpha Filtering**

**Goal**: To filter the initial universe of stocks and select only those predicted to have an upward trend in the next period.

This acts as a robust stock selection mechanism.

**Architecture**: A Convolutional Neural Network (CNN) processes 2D "image-like" representations of time-series data.

The inputs are multiple technical indicators (TIs) like RSI, MACD, and Stochastic Oscillators, calculated over a rolling window.

**Output**: The CNN classifies stocks into one of three classes (Rise, Fall, Hold).

Only stocks predicted to Rise are passed to Stage 2, significantly reducing the dimensionality of the subsequent optimization problem.

**Stage 2: LSTM-based Sharpe Ratio Maximization**

**Goal**: To determine the optimal allocation weights for the stocks selected in Stage 1.

This acts as the capital allocation mechanism.

**Architecture**: A Long Short-Term Memory (LSTM) network is used to capture sequential, multi-asset dependencies in the filtered stock data.

**Key solution** :

The model is trained using a custom loss function that directly minimizes the Negative Sharpe Ratio.

By minimizing $-\text{SR}$, the network is forced to learn portfolio weights that maximize the Sharpe Ratio, effectively making the network a policy network optimized for a financial metric rather than just prediction accuracy.

**Robust Backtesting: Walk-Forward Optimization (WFO)**

To ensure the simulation was realistic and free of look-ahead bias (a common pitfall in financial modeling), I employed a Walk-Forward Optimization (WFO) methodology:

A fixed-size Training Window (e.g., 252 days) is defined.

The models (CNN and LSTM) are trained only on the data within this window.

A single-step prediction and allocation is made for the very next day.

The window is advanced by one day, and the process repeats.

**WARNING**:

This notebook is not a financial advisor.

In [1]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=3c8173bb43c51a3522737efaad874b67a5039d2aab26ba1a8e57d37551639e78
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [2]:
import yfinance as yf
import pandas as pd
import ta as ta_lib
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Dropout, TimeDistributed
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

In [5]:


TICKERS_BY_COUNTRY = {
    'Germany': ['ALV.DE', 'DBK.DE', 'CBK.DE', 'HAG.DE', 'DB1.DE', 'FPE.DE', 'DHER.DE', 'MUV2.DE', 'VNA.DE', 'SDF.DE'],
    'France': ['BNP.PA', 'ACA.PA', 'GLE.PA', 'CS.PA', 'OR.PA', 'ENGI.PA', 'SCR.PA', 'CA.PA', 'PUB.PA', 'SAN.PA'],
    'UK': ['LLOY.L', 'BARC.L', 'HSBA.L', 'NWG.L', 'SSE.L', 'AV.L', 'PRU.L', 'LGEN.L', 'AHT.L', 'BP.L'],
    'Spain': ['SAN.MC', 'BBVA.MC', 'CABK.MC', 'AMS.MC', 'MAP.MC', 'SAB.MC', 'ELE.MC', 'ENG.MC', 'IBE.MC', 'IAG.MC'],
    'Italy': ['ISP.MI', 'UCG.MI', 'BAMI.MI', 'BMED.MI', 'FBK.MI', 'G.MI', 'AZM.MI', 'PST.MI', 'RACE.MI', 'IP.MI'],
    'Netherlands': ['INGA.AS', 'ADYEN.AS', 'ABN.AS', 'WKL.AS', 'AD.AS', 'ASML.AS', 'HEIA.AS', 'TKWY.AS', 'KPN.AS'],
    'Sweden': ['NDA-SE.ST', 'SEB-A.ST', 'SHB-A.ST', 'SWED-A.ST', 'GETI-B.ST', 'VOLV-B.ST', 'AZN.ST', 'TELIA.ST', 'ESSITY-B.ST', 'ERIC-B.ST'],
    'Switzerland': ['UBSG.SW', 'VETN.SW', 'ZURN.SW', 'NESN.SW', 'SGSN.SW', 'CFR.SW', 'GIVN.SW', 'SREN.SW', 'NOVN.SW'],
    'Belgium': ['KBC.BR', 'ABI.BR', 'UCB.BR', 'ACKB.BR', 'SOLB.BR', 'TUB.BR', 'ELI.BR', 'WDP.BR', 'COLR.BR']
}

TICKERS = [ticker for sublist in TICKERS_BY_COUNTRY.values() for ticker in sublist]
START_DATE = '2021-01-01'
END_DATE = '2025-09-30'
WINDOW_SIZE = 30
FLAT_THRESHOLD = 0.005
NUM_CLASSES = 3




def get_stock_data(tickers, start, end):

    data = yf.download(tickers, start=start, end=end)
    ti_data = {}

    if isinstance(data.columns, pd.MultiIndex):
        actual_tickers = data.columns.get_level_values(1).unique()
    else:
        actual_tickers = tickers

    for ticker in actual_tickers:
        if isinstance(data.columns, pd.MultiIndex):
            try:
                df = data.xs(ticker, axis=1, level=1).copy()
            except KeyError:
                continue
        else:
            df = data.copy()
            df.columns = [col.lower() for col in df.columns]

        if df.empty or len(df) < (WINDOW_SIZE + 30):
            continue

        if isinstance(data.columns, pd.MultiIndex):
             df.columns = [col.lower() for col in df.columns]

        required_cols = ['high', 'low', 'close', 'volume']
        if not all(col in df.columns for col in required_cols):
            continue

        try:
            df['RSI'] = ta_lib.momentum.rsi(df['close'], window=14)
            macd_indicator = ta_lib.trend.MACD(close=df['close'])
            df['MACD'] = macd_indicator.macd()
            df['MACDh'] = macd_indicator.macd_diff()
            df['MACDs'] = macd_indicator.macd_signal()
            stoch = ta_lib.momentum.StochasticOscillator(df['high'], df['low'], df['close'])
            df['stoch_k'] = stoch.stoch()
            df['stoch_d'] = stoch.stoch_signal()
            df['EMA_20'] = ta_lib.trend.ema_indicator(df['close'], window=20)
        except Exception as e:
            continue

        keep_cols = ['close', 'RSI', 'MACD', 'MACDh', 'MACDs', 'stoch_k', 'stoch_d', 'EMA_20']
        df = df[[col for col in keep_cols if col in df.columns]]

        final_df = df.dropna()
        if final_df.empty or len(final_df) < WINDOW_SIZE:
            continue

        ti_data[ticker] = final_df

    return ti_data

def create_classification_target(df, window=1, threshold=0.005):

    df['future_return'] = df['close'].shift(-window) / df['close'] - 1
    def classify(ret):
        if ret > threshold:
            return 2 # Rise
        elif ret < -threshold:
            return 0 # Fall
        else:
            return 1 # Flat
    df['target'] = df['future_return'].apply(classify)
    return df.dropna()


def create_cnn_data(ti_datasets, stocks, window_size):

    X_cnn, y_cnn = [], []
    for ticker in stocks:
        df = ti_datasets.get(ticker)
        if df is None or df.empty or len(df) <= window_size:
            continue

        df = df.drop(columns=['close', 'future_return'], errors='ignore')

        feature_cols = [col for col in df.columns if col != 'target']
        if not feature_cols:
            continue

        features = df[feature_cols].values
        targets = df['target'].values
        n_features = features.shape[1]

        for i in range(len(df) - window_size):
            window = features[i:i + window_size]

            X_cnn.append(window.reshape(window_size, n_features, 1))
            y_cnn.append(targets[i + window_size - 1])

    X_cnn = np.array(X_cnn)
    y_cnn = to_categorical(np.array(y_cnn), num_classes=NUM_CLASSES)
    return X_cnn, y_cnn

def build_cnn_model(input_shape):

    if not isinstance(input_shape, tuple) or len(input_shape) < 3:
        return None

    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (2, 2), activation='relu'),
        MaxPooling2D((2, 1)),
        Flatten(),
        Dense(100, activation='relu'),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def cnn_filter_stocks(cnn_model, ti_datasets, stocks, window_size):
    selected_stocks = []
    if cnn_model is None:
        print("CNN model is not available for filtering. Skipping stock selection.")
        return []

    print("\n--- Filtering stocks based on trained CNN prediction ('Rise' class=2) ---")

    for ticker in stocks:
        df = ti_datasets.get(ticker)
        if df is None or df.empty or len(df) < window_size:
            continue

        df_features = df.drop(columns=['close', 'future_return', 'target'], errors='ignore')
        if df_features.empty:
            continue

        latest_features = df_features.tail(window_size).values
        if latest_features.ndim < 2 or latest_features.shape[1] == 0:
            continue

        n_features = latest_features.shape[1]

        X_latest = latest_features.reshape(1, window_size, n_features, 1)

        try:
            prediction = cnn_model.predict(X_latest, verbose=0)
            predicted_class = np.argmax(prediction[0])
        except Exception as e:
            print(f"Skipping {ticker}: CNN prediction failed. Error: {e}")
            predicted_class = 0

        if predicted_class == 2:
            selected_stocks.append(ticker)

    return selected_stocks



def negative_sharpe_ratio_loss(y_true, y_pred):

    exp_pred = K.exp(y_pred)
    weights = exp_pred / K.sum(exp_pred, axis=-1, keepdims=True)


    portfolio_return = K.sum(weights * y_true, axis=-1)


    mean_return = K.mean(portfolio_return)
    std_return = K.std(portfolio_return)


    sharpe_ratio = mean_return / (std_return + K.epsilon())


    return -sharpe_ratio

def create_lstm_data(ti_datasets, selected_stocks, window_size):

    if not selected_stocks:
        return np.array([]), pd.DataFrame(), None

    aligned_dfs = []
    for ticker in selected_stocks:

        df = ti_datasets[ticker].drop(columns=['close', 'target'], errors='ignore')

        df.columns = [f'{ticker}_{col}' for col in df.columns]
        aligned_dfs.append(df)


    combined_df = pd.concat(aligned_dfs, axis=1, join='inner').dropna()

    feature_cols = [col for col in combined_df.columns if 'future_return' not in col]
    return_cols = [f'{ticker}_future_return' for ticker in selected_stocks]

    X_full = combined_df[feature_cols].values
    y_full_returns_df = combined_df[return_cols]


    scaler = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler.fit_transform(X_full)

    X_lstm, y_lstm_indices = [], []
    for i in range(len(X_scaled) - window_size):

        X_lstm.append(X_scaled[i:i + window_size])

        y_lstm_indices.append(i + window_size)


    y_lstm_returns = y_full_returns_df.iloc[y_lstm_indices]

    return np.array(X_lstm), y_lstm_returns, scaler

def build_lstm_model(input_shape, output_size):

    model = Sequential([
        LSTM(units=100, return_sequences=False, input_shape=input_shape),
        Dense(50, activation='relu'),

        Dense(output_size, activation='linear')
    ])
    model.compile(optimizer='adam', loss=negative_sharpe_ratio_loss)
    return model



def calculate_metrics(returns):

    annualization_factor = 252


    cumulative_return = (1 + returns).prod() - 1


    annualized_return = (cumulative_return + 1)**(annualization_factor / len(returns)) - 1


    annualized_volatility = returns.std() * np.sqrt(annualization_factor)


    sharpe_ratio = annualized_return / (annualized_volatility + 1e-8)

    return annualized_return, annualized_volatility, sharpe_ratio, cumulative_return

def walk_forward_backtest(X_all, y_all_returns, initial_train_idx, train_window, walk_step, n_epochs=2):

    N_SAMPLES, N_TIMESTEPS, N_FEATURES_FLAT = X_all.shape
    n_stocks_per_day = y_all_returns.shape[1]

    lstm_model = build_lstm_model(X_all.shape[1:], n_stocks_per_day)


    daily_returns = []
    daily_dates = []


    start_loop_index = initial_train_idx

    print("\nStarting Walk-Forward Loop...")


    for i in range(start_loop_index, len(X_all), walk_step):

        train_end_idx = i
        train_start_idx = train_end_idx - train_window

        if train_start_idx < 0:
            print(f"Skipping prediction at index {i}: Not enough training data. Required {train_window} samples, but only {i} available.")
            continue


        X_train_wfo = X_all[train_start_idx : train_end_idx]
        y_train_wfo_returns = y_all_returns.iloc[train_start_idx : train_end_idx].values


        lstm_model.fit(
            X_train_wfo,
            y_train_wfo_returns,
            epochs=n_epochs,
            batch_size=32,
            verbose=0,
            shuffle=False
        )


        predict_idx = i

        if predict_idx >= len(X_all):
            break


        X_predict = X_all[predict_idx:predict_idx + walk_step]
        y_actual_return = y_all_returns.iloc[predict_idx:predict_idx + walk_step].values
        current_date = y_all_returns.index[predict_idx]


        predicted_linear_output = lstm_model.predict(X_predict, verbose=0).flatten()


        exp_weights = np.exp(predicted_linear_output)
        final_weights = exp_weights / np.sum(exp_weights + K.epsilon())

        asset_returns = y_actual_return[0]


        daily_portfolio_return = np.dot(final_weights, asset_returns)


        daily_returns.append(daily_portfolio_return)
        daily_dates.append(current_date)

        if len(daily_returns) % 50 == 0:
             print(f"Processing... {len(daily_returns)} days backtested (Date: {current_date.strftime('%Y-%m-%d')})")



    return pd.Series(daily_returns, index=daily_dates)



K.clear_session()


ti_datasets = get_stock_data(TICKERS, START_DATE, END_DATE)
print(f"Successfully loaded data for {len(ti_datasets)} out of {len(TICKERS)} tickers.")


for ticker in TICKERS:
    if ticker in ti_datasets and not ti_datasets[ticker].empty:
        ti_datasets[ticker] = create_classification_target(ti_datasets[ticker], threshold=FLAT_THRESHOLD)


valid_tickers = [t for t in TICKERS if t in ti_datasets and not ti_datasets[t].empty and 'target' in ti_datasets[t].columns]
X_cnn, y_cnn = create_cnn_data(ti_datasets, valid_tickers, WINDOW_SIZE)
print(f"\nCNN Input Shape: {X_cnn.shape}, CNN Target Shape: {y_cnn.shape}")


if len(X_cnn) > 0:
    train_size_cnn = int(len(X_cnn) * 0.8)
    X_train_cnn, y_train_cnn = X_cnn[:train_size_cnn], y_cnn[:train_size_cnn]
    cnn_model = build_cnn_model(X_train_cnn.shape[1:])


    print("\n--- Training CNN Model (2 Epochs for quick run) ---")
    cnn_model.fit(X_train_cnn, y_train_cnn, epochs=2, batch_size=32, verbose=0)
else:
    print("No CNN data generated. Skipping CNN model building and filtering.")
    cnn_model = None


selected_stocks = cnn_filter_stocks(cnn_model, ti_datasets, valid_tickers, WINDOW_SIZE)
print(f"\nStocks selected by CNN filter ({len(selected_stocks)}): {selected_stocks}")


X_lstm_full, y_lstm_returns_full, lstm_scaler = create_lstm_data(ti_datasets, selected_stocks, WINDOW_SIZE)

if len(selected_stocks) > 0 and len(X_lstm_full) > 0:
    N_SAMPLES = X_lstm_full.shape[0]
    n_assets = len(selected_stocks)


    TRAIN_RATIO = 0.8
    N_INITIAL_TRAIN = int(TRAIN_RATIO * N_SAMPLES)
    TRAIN_WINDOW_SIZE = 252
    WALK_STEP = 1

    print(f"\nLSTM Input Shape (X_filtered): {X_lstm_full.shape}, LSTM Target Shape (y_filtered_returns): {y_lstm_returns_full.shape}")

    print("\n" + "="*50)
    print("     WFO CONFIGURATION AND INITIAL CHECK")
    print("="*50)
    print(f"Total days available for LSTM: {N_SAMPLES}")
    print(f"Initial Train Days (Static/CNN): {N_INITIAL_TRAIN}")
    print(f"WFO Rolling Train Window Size: {TRAIN_WINDOW_SIZE} days")
    print(f"WFO Backtest Period (Max Predictions): {N_SAMPLES - N_INITIAL_TRAIN}") # Corrected calculation
    print("="*50)


    if N_INITIAL_TRAIN < TRAIN_WINDOW_SIZE:
        print(f"Warning: N_INITIAL_TRAIN ({N_INITIAL_TRAIN}) is less than TRAIN_WINDOW_SIZE ({TRAIN_WINDOW_SIZE}). Adjusting N_INITIAL_TRAIN.")
        N_INITIAL_TRAIN = TRAIN_WINDOW_SIZE




    portfolio_returns_wfo = walk_forward_backtest(
        X_lstm_full,
        y_lstm_returns_full,
        N_INITIAL_TRAIN,
        TRAIN_WINDOW_SIZE,
        WALK_STEP,
        n_epochs=2
    )


    if not portfolio_returns_wfo.empty:
        ann_ret, ann_vol, sharpe, cum_ret = calculate_metrics(portfolio_returns_wfo)

        print("\n" + "="*50)
        print("        FINAL WALK-FORWARD BACKTEST RESULTS")
        print("="*50)
        print(f"Cumulative Return:        {cum_ret * 100:.2f}%")
        print(f"Annualized Return:        {ann_ret * 100:.2f}%")
        print(f"Annualized Volatility:    {ann_vol * 100:.2f}%")
        print(f"Sharpe Ratio (Rf=0):      {sharpe:.4f}")
        print(f"Backtest Period:          {portfolio_returns_wfo.index[0].strftime('%Y-%m-%d')} to {portfolio_returns_wfo.index[-1].strftime('%Y-%m-%d')}")
        print("="*50)
    else:
        print("\nNo portfolio returns generated during walk-forward backtest. Check WFO configuration or data availability.")

else:
    print("\n Not enough data or selected stocks to proceed with LSTM WFO optimization.")


/tmp/ipython-input-4119283787.py:26: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start, end=end)
[*********************100%***********************]  87 of 87 completed


Successfully loaded data for 87 out of 87 tickers.

CNN Input Shape: (88590, 30, 7, 1), CNN Target Shape: (88590, 3)

--- Training CNN Model (2 Epochs for quick run) ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



--- Filtering stocks based on trained CNN prediction ('Rise' class=2) ---

Stocks selected by CNN filter (85): ['ALV.DE', 'DBK.DE', 'CBK.DE', 'HAG.DE', 'DB1.DE', 'FPE.DE', 'DHER.DE', 'MUV2.DE', 'VNA.DE', 'SDF.DE', 'BNP.PA', 'ACA.PA', 'GLE.PA', 'CS.PA', 'OR.PA', 'ENGI.PA', 'SCR.PA', 'CA.PA', 'PUB.PA', 'SAN.PA', 'LLOY.L', 'BARC.L', 'HSBA.L', 'NWG.L', 'SSE.L', 'AV.L', 'PRU.L', 'LGEN.L', 'AHT.L', 'BP.L', 'SAN.MC', 'BBVA.MC', 'CABK.MC', 'AMS.MC', 'MAP.MC', 'SAB.MC', 'ELE.MC', 'ENG.MC', 'IBE.MC', 'IAG.MC', 'ISP.MI', 'UCG.MI', 'BAMI.MI', 'BMED.MI', 'FBK.MI', 'G.MI', 'AZM.MI', 'PST.MI', 'RACE.MI', 'IP.MI', 'INGA.AS', 'ADYEN.AS', 'ABN.AS', 'WKL.AS', 'AD.AS', 'ASML.AS', 'HEIA.AS', 'TKWY.AS', 'KPN.AS', 'NDA-SE.ST', 'SEB-A.ST', 'SHB-A.ST', 'SWED-A.ST', 'GETI-B.ST', 'VOLV-B.ST', 'AZN.ST', 'TELIA.ST', 'ESSITY-B.ST', 'ERIC-B.ST', 'UBSG.SW', 'VETN.SW', 'ZURN.SW', 'SGSN.SW', 'GIVN.SW', 'SREN.SW', 'NOVN.SW', 'KBC.BR', 'ABI.BR', 'UCB.BR', 'ACKB.BR', 'SOLB.BR', 'TUB.BR', 'ELI.BR', 'WDP.BR', 'COLR.BR']

L

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Starting Walk-Forward Loop...
Processing... 50 days backtested (Date: 2024-12-18)
Processing... 100 days backtested (Date: 2025-04-28)

        FINAL WALK-FORWARD BACKTEST RESULTS
Cumulative Return:        8.40%
Annualized Return:        17.65%
Annualized Volatility:    19.47%
Sharpe Ratio (Rf=0):      0.9063
Backtest Period:          2024-10-10 to 2025-09-26
